In [1]:
import numpy as np
import cv2 as cv
import mediapipe as mp

IP_CAMERA_URL = "http://192.168.1.78:8080/video"


cap = cv.VideoCapture(IP_CAMERA_URL)

# Initialize MediaPipe Hands and Drawing modules
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Define landmark indices for key points in the thumb
THUMB_TIP = 4
THUMB_MCP = 2

if not cap.isOpened():
    print("Cannot open camera")
    exit()

# Initialize the MediaPipe Hands module
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        # Convert the BGR frame to RGB as required by MediaPipe
        image_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        
        # Process the frame and recognize hands
        results = hands.process(image_rgb)
        
        # Initialize thumbs-up check flag
        thumbs_up = False
        
        # If hands are detected, draw landmarks on the frame
        hand_count = 0
        if results.multi_hand_landmarks:
            hand_count = len(results.multi_hand_landmarks)
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS)
                # Get thumb tip and thumb MCP coordinates
                thumb_tip = hand_landmarks.landmark[THUMB_TIP]
                thumb_mcp = hand_landmarks.landmark[THUMB_MCP]
                
                # For thumbs-up, the thumb tip should be higher than thumb MCP
                if thumb_tip.y > thumb_mcp.y:  # because higher y value is lower on image
                    thumbs_up = True
        if thumbs_up:
            cv.putText(frame, 'Thumbs Up!', (10, 60), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv.LINE_AA)
        # Overlay the hand count on the frame
        cv.putText(frame, f'Hands detected: {hand_count}', (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv.LINE_AA)
        cv.putText(frame, f'Thumb TIP: {hand_landmarks.landmark[THUMB_TIP]}', (10, 60), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv.LINE_AA)
        cv.putText(frame, f'Thumb MCP: {hand_landmarks.landmark[THUMB_MCP]}', (10, 90), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv.LINE_AA)

        # Display the frame with hand landmarks
        cv.imshow('Hand Recognition', frame)
        
        if cv.waitKey(1) == ord('q'):
            break

# Release the capture and destroy all OpenCV windows
cap.release()
cv.destroyAllWindows()

NameError: name 'hand_landmarks' is not defined